In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgbm
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

file_list=[]
file_list_train=[]
file_list_test=[]

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_list.append(os.path.join(dirname, filename))

PATH = '/kaggle/input/predict-volcanic-eruptions-ingv-oe/'

for dirname, _, filenames in os.walk('/kaggle/input/predict-volcanic-eruptions-ingv-oe/train'):
    for filename in filenames:
        file_list_train.append(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk('/kaggle/input/predict-volcanic-eruptions-ingv-oe/test'):
    for filename in filenames:
        file_list_test.append(os.path.join(dirname, filename))
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [ ]:
print(file_list[0])
print(pd.read_csv(file_list[0]))

In [ ]:
print(file_list_train[0])
print(pd.read_csv(file_list_train[0]))

In [ ]:
print(file_list[1])
print(pd.read_csv(file_list[1]))

In [ ]:
#file_list_test
print(len(file_list_test))
keys = list(pd.read_csv(file_list_test[0]).keys())
print(keys)
nanCount = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for index in range(len(file_list_test)):
    if(index % 200 == 0):
        print(index)
    df = pd.read_csv(file_list_test[index])
    for key in df.keys():
        if df[key].isna().sum() == 60001:
#             print(file_list_test[index])
#             print(keys.index(key))
            nanCount[keys.index(key)] += 1
#             print(key, 'is NAN!')
print(nanCount)
data={'sensors': keys, 'count': nanCount}
myFrame = pd.DataFrame(data)
print(myFrame)

    

In [ ]:
myFrame.plot(figsize =(20, 20), x="sensors", y="count", kind="bar",  rot=5, fontsize=14 )

In [ ]:
print(file_list_test[0])
print(pd.read_csv(file_list_test[0]))

In [ ]:
print(len(file_list_test))

In [ ]:
print(len(file_list_train))

In [ ]:
files_train = [file.split('/')[-1].split('.')[-2] for file in file_list_train]
files_test = [file.split('/')[-1].split('.')[-2] for file in file_list_test]
print(files_train[0:10])
print(files_test[0:10])


In [ ]:
test_set = set(files_test)
train_set = set(files_train)
inter = test_set.intersection(train_set)
print(inter)

In [ ]:
train = pd.read_csv(PATH+'train.csv')

In [ ]:
sns.distplot(train['time_to_eruption'], hist=True, kde=True, bins=100, color='blue', hist_kws={'edgecolor':'black'})

In [ ]:
train['time_to_eruption'].describe()

In [ ]:
df_segment_id = pd.read_csv(PATH+'/train/800654756.csv')

df_segment_id.plot(figsize=(20, 20), subplots=True, layout=(10, 1), rot = 0,
                  lw=1, title='segment_id #800654756')

In [ ]:
display(train.sort_values('time_to_eruption', axis=0, ascending=True).iloc[[0, -1], :])

segment_id_min = 601524801
segment_id_max = 1923243961

df_segment_id_min = pd.read_csv(PATH+'/train/'+str(segment_id_min)+'.csv')
df_segment_id_max = pd.read_csv(PATH+'/train/'+str(segment_id_max)+'.csv')

In [ ]:
df_segment_id_min.plot(figsize=(20,20), subplots=True, layout=(10,1), rot=0, lw=1, title='segment_id #601524801 (min)')
plt.show()

In [ ]:
df_segment_id_max.plot(figsize=(20,20), subplots=True, layout=(10,1), rot=0, lw=1, title='segment_id #1923243961 (min)')
plt.show()

In [ ]:
def build_features(signal, ts, sensor_id):
    X = pd.DataFrame()
    f = np.fft.fft(signal)
    f_real = np.real(f)
    X.loc[ts, f'{sensor_id}_sum'] = signal.sum()
    X.loc[ts, f'{sensor_id}_mean'] = signal.mean()
    X.loc[ts, f'{sensor_id}_std'] = signal.std()
    X.loc[ts, f'{sensor_id}_var'] = signal.var()
    X.loc[ts, f'{sensor_id}_max'] = signal.max()
    X.loc[ts, f'{sensor_id}_min'] = signal.min()
    X.loc[ts, f'{sensor_id}_skew'] = signal.skew()
    X.loc[ts, f'{sensor_id}_mad'] = signal.mad()
    X.loc[ts, f'{sensor_id}_kurtosis'] = signal.kurtosis()
    X.loc[ts, f'{sensor_id}_quantile99'] = np.quantile(signal, 0.99)
    X.loc[ts, f'{sensor_id}_quantile95'] = np.quantile(signal, 0.95)
    X.loc[ts, f'{sensor_id}_quantile85'] = np.quantile(signal, 0.85)
    X.loc[ts, f'{sensor_id}_quantile75'] = np.quantile(signal, 0.75)
    X.loc[ts, f'{sensor_id}_quantile55'] = np.quantile(signal, 0.55)
    X.loc[ts, f'{sensor_id}_quantile45'] = np.quantile(signal, 0.45)
    X.loc[ts, f'{sensor_id}_quantile25'] = np.quantile(signal, 0.25)
    X.loc[ts, f'{sensor_id}_quantile15'] = np.quantile(signal, 0.15)
    X.loc[ts, f'{sensor_id}_quantile05'] = np.quantile(signal, 0.05)
    X.loc[ts, f'{sensor_id}_quantile01'] = np.quantile(signal, 0.01)
    X.loc[ts, f'{sensor_id}_fft_real_mean'] = f_real.mean()
    X.loc[ts, f'{sensor_id}_fft_real_std'] = f_real.std()
    X.loc[ts, f'{sensor_id}_fft_real_max'] = f_real.max()
    X.loc[ts, f'{sensor_id}_fft_real_min'] = f_real.min()
    
    return X
   

In [ ]:
from itertools import islice


train_set = list()
seg=0
print(enumerate(train.segment_id))

for seg, segment_id in enumerate(train.segment_id):
    signals = pd.read_csv(PATH+'/train/'+str(segment_id)+'.csv')
    train_row=[]
#     print(segment_id)
    if seg % 200 == 0:
        print('Processing segment_id={}'.format(seg))
    
    for sensor in range(0, 10):
        sensor_id = f'sensor_{sensor+1}'
        train_row.append(build_features(signals[sensor_id].fillna(0), segment_id, sensor_id))
    
    train_row = pd.concat(train_row, axis=1)
    train_set.append(train_row)
    seg+=1
    
train_set = pd.concat(train_set)
print(train_set)

In [ ]:
train_set = train_set.reset_index()
train_set = train_set.rename(columns={'index': 'segment_id'})
print(train_set)
train_set = pd.merge(train_set, train, on='segment_id')
print(train_set)

In [ ]:
print(train_set.head(3))

In [ ]:
test_files = []
for dirname, _, filenames in os.walk(PATH+'/test/'):
    for filename in filenames:
        test_files.append(filename[:-4])

test = pd.DataFrame(test_files, columns=['segment_id'])

In [ ]:
test_set = list()
seg=0

for seg, segment_id in enumerate(test.segment_id):
    signals = pd.read_csv(PATH+'/test/'+str(segment_id)+'.csv')
    test_row=[]
    
    if seg%200 == 0:
        print('Processing segment_id={}'.format(seg))
        
    for sensor in range(0, 10):
        sensor_id = f'sensor_{sensor+1}'
        test_row.append(build_features(signals[sensor_id].fillna(0), segment_id, sensor_id))
    
    test_row = pd.concat(test_row, axis=1)
    test_set.append(test_row)
    seg+=1
    
test_set = pd.concat(test_set)
print(test_set)


In [ ]:
test_set=test_set.reset_index()
test_set=test_set.rename(columns={'index': 'segment_id'})
print(test_set)
test_set=pd.merge(test_set, test, on="segment_id")
print(test_set)

In [ ]:
print(test_set.head(3))

In [ ]:
X = train_set.drop(['segment_id', 'time_to_eruption'], axis=1)
y = train_set['time_to_eruption']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.head(3))
print('np.shape(X_train) = ', np.shape(X_train))

print(y_train.head(3))
print('np.shape(y_train) = ', np.shape(y_train))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=20, random_state=0)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_valid)

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_valid, y_pred)
fig=plt.figure()
mulreg = fig.add_subplot(1,1,1)
mulreg.scatter(y_valid, y_pred, color='r')
mulreg.set_title('Nonlinear Regression ')


In [ ]:
predictions = model.predict(test_set.drop(columns=['segment_id'], axis=1))

submission = pd.DataFrame()
submission['segment_id'] = test_set['segment_id']
submission['time_to_eruption'] = predictions
submission.to_csv('submission.csv', header=True, index=False)